# Influx v3
Desc: write metrics to Influx Database. 
   Read Metrics from a bucket   
This notebook is still in **DRAFT** mode and doesn't bring a lot of value. 

## Pre Req
1. Open an Iflux DB cloud account. Can be the free one. https://www.influxdata.com/get-influxdb/    
2. Create a new Bucket. For ex. Tables_bucket.
3. Open the "Load Data" option of "Client" 
4. Select "Python"
5. Follow the instructions. Step "install dependencies" is pip install influxdb3-python, pip install pandas

## Get Token. 
Without a token, the client code can't call the API

In [2]:
%%bash
export INFLUXDB_TOKEN=XE9AyZ-3y-HJNyupKWiLgzVo5JMew-Y31Vq7gbakekdP66wIkBslEdnyrCc-vQ0t9MGFj449z0LvFhepVOwFfw==

## Initialize Client
Copy the code. I named the organization in Influx "Dev"

Notice. The original code, proivded by Influx returns an error: ```[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)``` . To solve it, add the coomand ``` ssl_ca_cert=certifi.where()```. Resource: # https://stackoverflow.com/questions/69401104/influxdb-2-0-certificate-verify-failed-certificate-has-expired-ssl-c1129
   

In [3]:
import os, time
from influxdb_client_3 import InfluxDBClient3, Point
import certifi # we need it to support their certification problems. 

token = os.environ.get("INFLUXDB_TOKEN")
token = "XE9AyZ-3y-HJNyupKWiLgzVo5JMew-Y31Vq7gbakekdP66wIkBslEdnyrCc-vQ0t9MGFj449z0LvFhepVOwFfw=="
print (token)
org = "Dev"
host = "https://us-east-1-1.aws.cloud2.influxdata.com"

client = InfluxDBClient3(host=host, token=token, org=org, ssl_ca_cert=certifi.where())

XE9AyZ-3y-HJNyupKWiLgzVo5JMew-Y31Vq7gbakekdP66wIkBslEdnyrCc-vQ0t9MGFj449z0LvFhepVOwFfw==


## Copy Data 
The UI detects the existsing Bucket. 



In this data example, we have some important concepts:
- **measurement**: Primary filter for the thing you are measuring. Since we are measuring the sample census of insects, our measurement is "census".
- **tag**: Key-value pair to store metadata about your fields. We are storing the "location" of where each census is taken. Tags form part of your primary key.
- **field**:	Key-value pair that stores the actual data you are measuring.	We are storing the insect "species" and "count" as the key-value pair. Fields are not indexed and can be stored as integers, floats, strings, or booleans.

In [4]:
database="Tables_Bucket"

data = {
  "point1": {
    "location": "Klamath",
    "species": "bees",
    "count": 23,
  },
  "point2": {
    "location": "Portland",
    "species": "ants",
    "count": 30,
  },
  "point3": {
    "location": "Klamath",
    "species": "bees",
    "count": 28,
  },
  "point4": {
    "location": "Portland",
    "species": "ants",
    "count": 32,
  },
  "point5": {
    "location": "Klamath",
    "species": "bees",
    "count": 29,
  },
  "point6": {
    "location": "Portland",
    "species": "ants",
    "count": 40,
  },
}

for key in data:
  point = (
    Point("census")
    .tag("location", data[key]["location"])
    .field(data[key]["species"], data[key]["count"])
  )
  client.write(database=database, record=point)
  time.sleep(1) # separate points by 1 second

print("Complete. Return to the InfluxDB UI.")


Complete. Return to the InfluxDB UI.


## Execute a Simple Query
v3 supports SQL 

In [5]:
query = """SELECT *
FROM 'census'
WHERE time >= now() - interval '24 hours'
AND ('bees' IS NOT NULL OR 'ants' IS NOT NULL)"""

# Execute the query
table = client.query(query=query, database="Tables_Bucket", language='sql') 

# Convert to dataframe
df = table.to_pandas().sort_values(by="time")
print(df)




    ants  bees  location                          time
6    NaN  23.0   Klamath 2023-09-17 15:50:32.816591687
9   30.0   NaN  Portland 2023-09-17 15:50:34.151910341
7    NaN  28.0   Klamath 2023-09-17 15:50:35.335547781
10  32.0   NaN  Portland 2023-09-17 15:50:36.595085749
8    NaN  29.0   Klamath 2023-09-17 15:50:37.815063826
11  40.0   NaN  Portland 2023-09-17 15:50:38.975112553
0    NaN  23.0   Klamath 2023-09-17 19:48:09.639242780
3   30.0   NaN  Portland 2023-09-17 19:48:10.808677735
1    NaN  28.0   Klamath 2023-09-17 19:48:12.002052431
4   32.0   NaN  Portland 2023-09-17 19:48:13.156166514
2    NaN  29.0   Klamath 2023-09-17 19:48:14.321562173
5   40.0   NaN  Portland 2023-09-17 19:48:15.539494349


Advance Query
Sep 17 - doesn't work. Not sure why as it is copy paste from the demo

In [6]:
## Execute Aggregate Queries
query = """SELECT mean(count)
FROM 'census'
WHERE time > now() - '10m'"""

# Execute the query
table = client.query(query=query, database="Tables_Bucket", language='influxql') 

# Convert to dataframe
df = table.to_pandas().sort_values(by="time")
print(df)

ArrowInvalid: Flight returned invalid argument error, with message: Error while planning query: Error during planning: invalid FROM clause, expected identifier, regular expression or subquery at pos 24. gRPC client debug context: UNKNOWN:Error received from peer ipv4:18.214.223.180:443 {created_time:"2023-09-17T22:48:17.501071+03:00", grpc_status:3, grpc_message:"Error while planning query: Error during planning: invalid FROM clause, expected identifier, regular expression or subquery at pos 24"}. Client context: IOError: Server never sent a data message. Detail: Internal

## Delete a bucket


In [ ]:
import requests

# InfluxDB API endpoint
base_url = "http://localhost:8086"
org = "your_organization"  # Replace with your organization name
bucket = "your_bucket"      # Replace with the name of the bucket you want to delete

# Authentication token (if required)
token = "your_authentication_token"  # Replace with your authentication token, if needed

# Construct the URL for deleting the bucket
url = f"{base_url}/api/v2/buckets/{org}/{bucket}"

# Headers for the request (include the authentication token if required)
headers = {
    "Authorization": f"Token {token}" if token else "",
}

# Send the DELETE request to delete the bucket
response = requests.delete(url, headers=headers)

# Check the response status code
if response.status_code == 204:
    print(f"Bucket '{bucket}' deleted successfully.")
else:
    print(f"Failed to delete bucket '{bucket}'. Status code: {response.status_code}")
    print(response.text)
